In [13]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd
import pickle

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)


In [29]:
def strCont(string, x):
    return string.find(x) != -1

def setTech(x):
    if(strCont(x, 'react')):
        return 'react'
    elif(strCont(x, 'vue')):
        return 'vue'
    elif(strCont(x, 'angular')):
        return 'angular'
    elif(strCont(x, 'express')):
        return 'express'
    elif(strCont(x, 'django')):
        return 'django'
    elif(strCont(x, 'symfony')):
        return 'symfony'
    elif(strCont(x, 'ruby-on-rails')):
        return 'ruby-on-rails'
    elif(strCont(x, 'node')):
        return 'node.js'
    elif(strCont(x, 'asp')):
        return 'asp'
    elif(strCont(x, 'servlet') or strCont(x, 'jsp')):
        return 'java-servlet'
    elif(strCont(x, 'php')):
        return 'php'
    elif(strCont(x, 'html')):
        return 'html'
    elif(strCont(x, 'javascript') or strCont(x, 'js')):
        return 'javascript'
    elif(strCont(x, 'spring')):
        return 'spring'
    else:
        return 'None'

def creationDateToDay(date):
    return pd.to_datetime(date).to_period('M')

In [15]:
# Percent of questions on stack overflow relative to new and old technologies
# 1) from db take N questions, randomly, selecting tags and date.
# 2) create a new column named technlogy, based on tags 
# 3) substitute timestamp with month

In [32]:
#GET TAGS DATASET AND MODIFY

sample_size = 1000000 #--> 1 MLN

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE DATETIME(creation_date) < DATETIME(2020, 3,30,0,0,0)
    ORDER BY r
    LIMIT ''' + str(sample_size)

all_question = (
    client.query(query).result()
    .to_dataframe(bqstorage_client=storage_client)
)

print("Queried: ", (len(pickle.dumps(all_question)) + 8 * sample_size)/2**30, "GB")

all_question = all_question.drop(columns=['r']).rename(columns={'tags':'technology'})
all_question['technology']=all_question['technology'].map(setTech)
all_question['month']=all_question['creation_date'].map(creationDateToDay)
all_question = all_question.drop(columns=['creation_date'])
all_question

Queried:  0.04235075321048498 GB


/home/kalu/.local/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,technology,month
0,None,2015-02
1,None,2017-07
2,None,2015-11
3,None,2017-05
4,None,2016-02
...,...,...
999995,None,2017-10
999996,html,2018-05
999997,None,2018-08
999998,php,2014-05


In [33]:
#SAVE TO CSV
all_question.to_csv('tags_dataframe.csv')

In [24]:
#GET N ANSWER N QUESTION FOR USER

sample_size = 10000 #--> 100 k

'''

    SELECT rand() as r, users.id, 
        count(post_questions.id) as quest_count, 
        count(post_answers.id) as answ_count,
    from bigquery-public-data.stackoverflow.posts_questions as post_questions, 
        bigquery-public-data.stackoverflow.posts_answers as post_answers,
        bigquery-public-data.stackoverflow.users as users
    where users.id = post_questions.owner_user_id
    or users.id = post_answers.owner_user_id
    group by users.id
    order by r
    limit '''

query = '''
    WITH user_quest_count AS (
            SELECT users.id, users.reputation, count(post_questions.id) as quest_count, 
            from bigquery-public-data.stackoverflow.posts_questions as post_questions,
            bigquery-public-data.stackoverflow.users as users
            where users.id = post_questions.owner_user_id
            group by users.id, users.reputation
        ),
        user_answ_count AS (
            SELECT users.id, count(post_answers.id) as answ_count, 
            from bigquery-public-data.stackoverflow.posts_answers as post_answers,
            bigquery-public-data.stackoverflow.users as users
            where users.id = post_answers.owner_user_id
            group by users.id, users.reputation
        )
    SELECT RAND() as r, user_quest_count.quest_count, user_quest_count.id, user_quest_count.reputation, user_answ_count.answ_count
    FROM user_quest_count
    LEFT JOIN user_answ_count ON user_quest_count.id = user_answ_count.id
    order by r
    limit ''' + str(sample_size)


answers_score = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
    .drop(columns=['r'])
)

answers_score.fillna(0)

,quest_count,id,reputation,answ_count
0,2,11162759,23,1.0
1,5,2216547,15,0.0
2,2,9217138,23,0.0
3,1,6587176,1,0.0
4,1,11344172,27,0.0
...,...,...,...,...
9995,1,3214202,1,0.0
9996,1,9021605,3,0.0
9997,1,7527289,1,0.0
9998,29,10389241,109,6.0


In [46]:
answers_score.fillna(0).to_csv('ans_quest_dataframe.csv')

In [33]:
answers_score.sort_values('quest_count')

,id,quest_count,answ_count
0,1379657,1,1
66671,75642,1,1
66670,5384363,1,1
66669,6241235,1,1
66668,2899889,1,1
...,...,...,...
33329,4607317,1,1
33328,2598316,1,1
33327,5046896,1,1
33349,832621,1,1
